In [1]:
#In this notebook machine learning is used to solve one interseting problem.
#we are going to create a programm, which is able to guesse to which countries cuisine belongs the receipe
#according to its ingredients. It is quite a natural problem, because the ingredients are strongly connected with
#the country to which cuisine the food belongs. Every expert cook can do such kind of classification, Now we want the
#computer to be as smart as an expert cook in this problem.

#
import json 
import pandas as pd
import numpy as np
import nltk

from collections import defaultdict

from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.metrics import accuracy_score

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

In [2]:
with open('train.json') as f:
     data = pd.DataFrame(json.load(f))

In [3]:
def process_ingredients(ingredient_list):
    
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    
    separator = ' '
    tokenized_list = word_tokenize(separator.join(ingredient_list).lower())
    
    final_words = []
    word_Lemmatized = WordNetLemmatizer()
    
    for word, tag in pos_tag(tokenized_list):
        if word not in stopwords.words('english') and word.isalpha():
            final_word = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            final_words.append(final_word)
            
    return separator.join(final_words)

In [4]:
data['ingredients'] = data['ingredients'].apply(process_ingredients)

In [5]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['ingredients'],data['cuisine'],test_size=0.3)

label_encoder = LabelEncoder()

train_y_encoded = label_encoder.fit_transform(train_y)
test_y_encoded = label_encoder.transform(test_y)

In [6]:
tfidf_vect = TfidfVectorizer(max_features=6000)

tfidf_vect.fit(train_x)
train_x_tfidf = tfidf_vect.transform(train_x)
test_x_tfidf = tfidf_vect.transform(test_x)

In [13]:
svm = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm.fit(train_x_tfidf,train_y_encoded)
predictions_svm = svm.predict(test_x_tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_svm, test_y_encoded)*100)

SVM Accuracy Score ->  78.0273191988603


In [27]:
svm_rbf = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma=1.5)
svm_rbf.fit(train_x_tfidf,train_y_encoded)
predictions_svm_rbf = svm_rbf.predict(test_x_tfidf)
print("SVM with rbf kernel Accuracy Score -> ",accuracy_score(predictions_svm_rbf, test_y_encoded)*100)

SVM with rbf kernel Accuracy Score ->  79.24243693957932


In [26]:
knn = KNeighborsClassifier(n_neighbors = 11, weights = 'distance')
knn.fit(train_x_tfidf[0:2000],train_y_encoded[0:2000])
predictions_knn = knn.predict(test_x_tfidf)
print("KNN Accuracy Score -> ",accuracy_score(predictions_knn, test_y_encoded)*100)

KNN Accuracy Score ->  65.79234056817229
